TensorFlow 'raw' :: MNIST MLP
==============================

This is a quick illustration of a single-layer network training on the MNIST data.

( Credit for the original workbook : Aymeric Damien :: https://github.com/aymericdamien/TensorFlow-Examples )

In [ ]:
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

import gzip
import pickle

In [ ]:
# Seed for reproducibility
np.random.seed(42)

### Get the MNIST data
Put it into useful subsets, and show some of it as a sanity check

In [ ]:
# Download the MNIST digits dataset (only if not present locally)
import os
import urllib.request 

mnist_data = './data/MNIST' 
mnist_pkl_gz = mnist_data+'/mnist.pkl.gz'

if not os.path.isfile(mnist_pkl_gz):
    if not os.path.exists(mnist_data):
        os.makedirs(mnist_data)
    print("Downloading MNIST data file")
    urllib.request.urlretrieve(
        'http://deeplearning.net/data/mnist/mnist.pkl.gz', 
        mnist_pkl_gz)

print("MNIST data file available locally")

In [ ]:
# Load training and test splits as numpy arrays
train, val, test = pickle.load(gzip.open(mnist_pkl_gz), encoding='iso-8859-1')

X_train, y_train = train
X_val, y_val = val

In [ ]:
# The original 28x28 pixel images are flattened into 784 dimensional feature vectors
X_train.shape

In [ ]:
# Plot the first few examples 
plt.figure(figsize=(12,3))
for i in range(10):
    plt.subplot(1, 10, i+1)
    plt.imshow(X_train[i].reshape((28, 28)), cmap='gray', interpolation='nearest')
    plt.axis('off')

### Create the Network


In [ ]:
# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input],   name='x_input')
#y = tf.placeholder("int32", [None, n_classes], name='y_target')  # originally, a one-hot label index
y = tf.placeholder("int32", [None, 1], name='y_target')  # This is the label index

In [ ]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [ ]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
logits = multilayer_perceptron(x, weights, biases)
pred = tf.argmax(logits, axis=1)

In [ ]:
# Define an 'op' that initializes the variables
init = tf.global_variables_initializer()

### Set up the Loss Function
So that we can perform Gradient Descent to improve the networks' parameters during training

In [ ]:
# Define optimizer for the labels (expressed as a onehot encoding)
labels = tf.one_hot(indices=y, depth=10)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))

### Set up the Training Function


In [ ]:
# Parameters for the training phase
learning_rate = 0.001
training_epochs = 10
BATCH_SIZE = 100
display_step = 1

In [ ]:
# Define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

### Batching of Training
For efficiency, we operate on data in batches, so that (for instance) a GPU can operate on multiple examples simultaneously

In [ ]:
# We'll choose a batch size, and calculate the number of batches in an "epoch"
# (approximately one pass through the data).
N_BATCHES = len(X_train) // BATCH_SIZE
N_VAL_BATCHES = len(X_val) // BATCH_SIZE

In [ ]:
# For training, we want to sample examples at random in small batches
def batch_gen(X_, y_, N):
    while True:
        idx = np.random.choice(len(y_), N)
        yield X_[idx].astype('float32'), np.expand_dims(y_[idx].astype('int32'), 1)

In [ ]:
# Minibatch generators for the training and validation sets
train_batches = batch_gen(X_train, y_train, BATCH_SIZE)
val_batches = batch_gen(X_val, y_val, BATCH_SIZE)

In [ ]:
# Try sampling from the batch generator.
# Plot an image and corresponding label to verify they match.
X0, y0 = next(train_batches)
plt.imshow(X0[0].reshape((28, 28)), cmap='gray', interpolation='nearest')
print(y0[0])

### Finally, the Training...
For each epoch, we call the training function N_BATCHES times, accumulating an estimate of the training loss and accuracy.

Then we do the same thing for the validation set.

We print out the ratio of loss in the validation set vs the training set to help recognize overfitting.

In [ ]:
#for epoch in range(10):
#    train_loss = 0
#    train_acc = 0
#    for _ in range(N_BATCHES):
#        X, y = next(train_batches)
#        loss, acc = f_train(X, y)
#        train_loss += loss
#        train_acc += acc
#    train_loss /= N_BATCHES
#    train_acc /= N_BATCHES
#
#    val_loss = 0
#    val_acc = 0
#    for _ in range(N_VAL_BATCHES):
#        X, y = next(val_batches)
#        loss, acc = f_val(X, y)
#        val_loss += loss
#        val_acc += acc
#    val_loss /= N_VAL_BATCHES
#    val_acc /= N_VAL_BATCHES
#    
#    print('Epoch {:2d}, Train loss {:.03f}     (validation : {:.03f}) ratio {:.03f}'.format(
#            epoch, train_loss, val_loss, val_loss/train_loss))
#    print('          Train accuracy {:.03f} (validation : {:.03f})'.format(train_acc, val_acc))
#print("DONE")

In [ ]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)  # Running this 'op' initialises the network weights

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        
        # Loop over all batches
        for batch_x, batch_y in train_batches:
            print(batch_x.shape, batch_y.shape)
            
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
            
            # Compute average loss
            avg_cost += c / N_BATCHES
            
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=","{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))
print("DONE")

### Visualising the Weight Matrix
We can retrieve the value of the trained weight matrix from the output layer.

It can be interpreted as a collection of images, one per class.

In [ ]:
#weights = l_out.W.get_value()
#print(weights.shape)

### Plot the weight images.  
We should expect to recognize similarities to the target images:

In [ ]:
plt.figure(figsize=(12,3))
for i in range(10):
    plt.subplot(1, 10, i+1)
    plt.imshow(weights[:,i].reshape((28, 28)), cmap='gray', interpolation='nearest')
    plt.axis('off')

Exercises
=====

1. Logistic regression
----------------------

The simple network we created is similar to a logistic regression model. Verify that the accuracy is close to that of `sklearn.linear_model.LogisticRegression`.

In [ ]:
# Uncomment and execute this cell for an example solution
# %load ../models/spoilers/logreg.py

2. Hidden layer
---------------

Try adding one or more "hidden" `DenseLayers` between the input and output. Experiment with different numbers of hidden units.

In [ ]:
# Uncomment and execute this cell for an example solution
# %load ../models/spoilers/hiddenlayer.py

3. Optimizer
------------

Try one of the other algorithms available in `lasagne.updates`. You may also want to adjust the learning rate.
Visualize and compare the trained weights. Different optimization trajectories may lead to very different results, even if the performance is similar. This can be important when training more complicated networks.

In [ ]:
# Uncomment and execute this cell for an example solution
# %load ../models/spoilers/optimizer.py